The Z Algorithm is an algorithm to find exact matches in a string. When analyzing runtimes, let the length of the pattern be $n$ and the length of the full text be $m$, $m >> n$. 

**TODO:** 
- Test runtime for different values of n and m, should be linear, unlike naive. Well naive is only quadratic when lots of inner for loops run, so need lots of prefixes of pattern throughout string. 
  - If this works, it means there's no hidden quadratic string behavior
- Move to a Python file
- More doctests
- Test on larger RNA files 

In [64]:
from typing import List, Optional
from string import ascii_letters, digits
from random import choices
from time   import time

In [58]:
possible_chars = ascii_letters + digits + " "

In [81]:
from doctest import testmod
testmod()

TestResults(failed=0, attempted=22)

## Naive algorithm

In [41]:
def naive_match(pattern: str, text: str) -> Optional[int]: 
    """Return index of start of match. O(mn) time. 
    
    >>> naive_match("Hello", "Hello world")
    0
    >>> naive_match("world", "Hello world")
    6
    >>> naive_match("", "123")
    0
    >>> naive_match("12", "")
    >>> naive_match("", "")
    0
    """
    for i in range(len(text) - len(pattern) + 1):
        for j, c in enumerate(pattern):
            if c != text[i + j]:
                break
        else: 
            return i
    return None

To improve this runtime, we need to be able to look at each character only a constant number of times. We've got to be able to make use of the fact that while comparing at an earlier position, we might have seen later characters.

The Z Algorithm finds the longest length substring at each index $i$ of a string that matches a prefix of the string. 

A **Z-value** $Z_i(S)$ is the length of the longest prefix of $S$ starting at index $i$. 

A **Z-box** for $Z_i (S) > 0$ is the substring of $S$ from $i$ to $i + Z_i (S) - 1$ (a prefix of $S$). 

In [59]:
def prefix_match_len(s1: str, s2: str) -> int: 
    """Return length of longest prefix match of s1 and s2.
    
    >>> prefix_match_len("", "")
    0
    >>> prefix_match_len("Hello there", "Bye")
    0
    >>> prefix_match_len("Hello there", "Hello ")
    6
    >>> prefix_match_len("Hello there", "Hello there")
    11
    """
    for i, (c1, c2) in enumerate(zip(s1, s2)):
        if c1 != c2: 
            return i
    return min(len(s1), len(s2))
        
def Z_scores(text: str) -> List[int]:
    """Compute Z scores. (0th Z score is set to 0. Technically, it's
    the full length.) O(m + n) time. 
    
    Track the left and right indices (l, r) of the rightmost Z-box 
    after each iteration. 
    First, manually compute Z_2 via character comparison. 
    
    Notice r tells us the farthest we've seen (since we break if a 
    character doesn't match). If we're checking the Z-score of an 
    index i > r, we have no information so we've got to do it 
    manually. 
    
    Otherwise, we need to use the information we have. 
    Compute beta = r - i + 1, which is the amount of rightmost Z-box 
    left. This is how many more characters we've seen. 
    
    Here's the core trick of the algorithm. We know the substring 
    from l to r is a prefix. Now compute j = i - l + 1, which is the 
    start index near the beginning of the entire string. It's very 
    close to the beginning and we know it matches i because i is part 
    of a prefix. 
        
    Case 1: Z_j < beta. That means Z_i = Z_j because we know up to beta 
    is a prefix, but Z_j tells us that its prefix doesn't go that far. 
    
    Case 2: Z_j > beta. That means Z_i = beta because Z_j matches a lot 
    more, but we know our match can only go up to beta because past that 
    isn't a prefix. 
    
    Case 3: Z_j == beta. That means Z_i >= Z_j, so we've got to do the 
    rest by character comparison.
    
    Finally, update l and r if need be. 
    
    >>> Z_scores("HelloXHello world")
    [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    >>> Z_scores("ABCDBCD")
    [0, 0, 0, 0, 0, 0, 0]
    >>> Z_scores("HelloXHello HelloH")
    [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1]
    """
    l = r = 0
    Z_scores = [0] 
    
    # Compute Z_1 (zero-indexed) manually 
    Z_1 = prefix_match_len(text[1:], text)
    if Z_1: 
        l, r = 1, 1 + Z_1 - 1
    Z_scores.append(Z_1)
    
    for i in range(2, len(text)): 
        if i > r: 
            # No information so use character comparison
            Z_i = prefix_match_len(text[i:], text)
        else: 
            beta = r - i + 1
            j = i - l + 1

            if Z_scores[j] < beta: 
                Z_i = Z_scores[j] 
            elif Z_scores[j] > beta:
                Z_i = beta
            else: 
                Z_i = beta # or Z_scores[j] since they're equivalent 
                Z_i += prefix_match_len(text[i + Z_i:], text[Z_i:])
        
        Z_scores.append(Z_i)
        if i + Z_i - 1 > r: 
            l, r = i, i + Z_i - 1
    return Z_scores


    

def Z_match(pattern: str, text: str) -> Optional[int]:
    """Use the Z algorithm to find pattern in text.
    
    >>> Z_match("Hello", "Hello world")
    0
    >>> Z_match("world", "Hello world")
    6
    >>> Z_match("", "123")
    0
    >>> Z_match("12", "")
    >>> Z_match("", "")
    0
    """
    if not pattern: 
        return 0

    # Need to find a character not used in pattern or text, to act as a 
    # divider. 
    unused_chars = set(possible_chars) - set(pattern) - set(text)
    if unused_chars:
        divider = unused_chars.pop()
    else: 
        raise ValueError("Need at least one unused ascii letter, digit, or space in pattern and string!")
    
    for i, Z_score in enumerate(Z_scores(pattern + divider + text)):
        if Z_score == len(pattern):
            return i - len(pattern) - 1
    return None

In [144]:
def modified_Z_match(text: str, pattern_len: int) -> Optional[int]:
    """
    Largely the same Z_scores algorithm as above except: 
    
    There's a huge space overhead with storing the entire list. But 
    notice we don't need to store the entire list if the text is of 
    the form P$T, where $ is an unused character in P and T. The 
    prefix can be at most |P| length, so we only need to store the 
    Z-scores of first |P| indices, and then once we find a Z-score of 
    |P|, we can just return that. 
    
    There's also a large space overhead with copying the string each
    time we call prefix_match_len. To fix this, I use memoryview, which 
    *dramatically* improves runtime. 
    
    >>> modified_Z_match("HelloXHello world", 5)
    0
    >>> modified_Z_match("worldXHello world", 5)
    6
    >>> modified_Z_match("123", 0)
    0
    >>> modified_Z_match("12X", 2)
    >>> modified_Z_match("", 0)
    0
    """
    if not pattern_len: 
        return 0
    
    l = r = 0
    Z_scores = [0]
    
    text = memoryview(bytes(text, encoding='utf-8'))
    appends = 0
    
    # Compute Z_1 (zero-indexed) manually 
    Z_1 = prefix_match_len(text[1:], text)
    if Z_1: 
        l, r = 1, 1 + Z_1 - 1
    Z_scores.append(Z_1)
    
    for i in range(2, len(text)): 
        if i > r: 
            # No information so use character comparison
            Z_i = prefix_match_len(text[i:], text)
        else: 
            beta = r - i + 1
            j = i - l + 1

            if Z_scores[j] < beta: 
                Z_i = Z_scores[j] 
            elif Z_scores[j] > beta:
                Z_i = beta
            else: 
                Z_i = beta # or Z_scores[j] since they're equivalent 
                Z_i += prefix_match_len(text[i + Z_i:], text[Z_i:])
        
        # Are we done? 
        if Z_i == pattern_len:
            return i - pattern_len - 1
        
        # Only append if i < pattern_len 
        if i < pattern_len:
            appends += 1
            Z_scores.append(Z_i)
        if i + Z_i - 1 > r: 
            l, r = i, i + Z_i - 1
    return None

## Simple timing test

In [152]:
main_repeat = "".join(choices(possible_chars, k=10)) # Want there to be unused char to use as divider
pattern = main_repeat * 1_000
# Note the Z algorithm is only different from the naive one if prefixes of the pattern are found n
# throughout the text
text = "".join([main_repeat * 500 + "asdfghjkljhgfs"] * 100) + pattern

unused_chars = set(possible_chars) - set(pattern) - set(text)
if unused_chars:
    divider = unused_chars.pop()
else: 
    raise ValueError("Need at least one unused ascii letter, digit, or space in pattern and string!")
combined = pattern + divider + text

In [153]:
# Test naive 
start = time() 
start_index = naive_match(pattern, text)
end = time() 
print(f"Start index is {start_index}. It took {end - start} seconds to compute that.")

Start index is 501400. It took 10.4326012134552 seconds to compute that.


In [154]:
import cProfile
cProfile.run("modified_Z_match(combined, len(pattern))")

         11520 function calls in 0.214 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.172    0.172    0.214    0.214 <ipython-input-144-80826a9802c5>:1(modified_Z_match)
     1512    0.041    0.000    0.041    0.000 <ipython-input-59-ce46e4e3671e>:1(prefix_match_len)
        1    0.000    0.000    0.214    0.214 <string>:1(<module>)
        1    0.000    0.000    0.214    0.214 {built-in method builtins.exec}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.min}
     9999    0.001    0.000    0.001    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [155]:
# Test Z match 
start = time() 
start_index = modified_Z_match(combined, len(pattern))
end = time() 
print(f"Start index is {start_index}. It took {end - start} seconds to compute that.")

Start index is 501400. It took 0.2177278995513916 seconds to compute that.
